# Used packagages

In [1]:
import pandas as pd
import numpy as np
import re

# Upload data

In [2]:
cars = pd.read_csv('used_cars.csv')
cars.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


## Potential cleaning issues to explore

•Lack of uniformity in brand name (INIFINITY is all caps, while other brands are standard style. <br>
•Model and trim should be separated into distinct columns.<br>
•Remove 'mi' from milage. Change spelling of column name to 'mileage'. <br>
•Explore fuel types. Standardize as needed. <br>
•Explore enginge types. Standardize as needed. <br>
•Transmission needs to be automatic or manual. Potentially add additional column for number of speeds? <br>
•Explore accident. How many classifications are there? Can it be made into a yes/no or does it need more detail? <br>
•NaNs show up in clean_title. How to handle them? <br>
•Price is the target

## Initial data exploration

In [3]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


There are nulls to be dealt with in fuel_type, accident, and clean_title. <b>Determine a strategy to handle these values.</b>

In [4]:
#rename milage to mileage
cars.rename(columns ={'milage' : 'mileage'}, inplace=True)

#remove $ from price column
cars['price'] = cars['price'].str.replace('$', '')
#remove , from price column
cars['price'] = cars['price'].str.replace(',', '')
#remove , and 'mi.' from miles
cars['mileage'] = cars['mileage'].str.replace(',', '')
cars['mileage'] = cars['mileage'].str.replace('mi.', '')

#change mileage and price to int64
convert_dict = {'mileage' : int, 'price' : int}
cars = cars.astype(convert_dict)

#check with info
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   mileage       4009 non-null   int64 
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   int64 
dtypes: int64(3), object(9)
memory usage: 376.0+ KB


## Exploring columns for cleaning

### Brand

In [5]:
#Count values. Sort by index to view potential duplicates more easily.b
cars['brand'].value_counts().sort_index()

brand
Acura             64
Alfa              19
Aston              9
Audi             200
BMW              375
Bentley           33
Bugatti            1
Buick             30
Cadillac         107
Chevrolet        292
Chrysler          28
Dodge             90
FIAT               5
Ferrari           12
Ford             386
GMC               91
Genesis           20
Honda             63
Hummer            16
Hyundai           72
INFINITI          59
Jaguar            47
Jeep             143
Karma              2
Kia               76
Lamborghini       26
Land             130
Lexus            163
Lincoln           52
Lotus              4
Lucid              3
MINI              33
Maserati          34
Maybach            1
Mazda             64
McLaren            6
Mercedes-Benz    315
Mercury            3
Mitsubishi        20
Nissan           116
Plymouth           1
Polestar           1
Pontiac           15
Porsche          201
RAM               91
Rivian            17
Rolls-Royce       11
Saab   

It appears that some car brands deploy upper and lowercase purposefully as part of their brands. As a result, we will not need to clean this column further.

### Model

In [6]:
len(pd.unique(cars['model']))

1898

In [7]:
cars['model'].value_counts().sort_index()

model
124 Spider Abarth          1
128 i                      1
135 i                      3
135 is                     1
1500 Big Horn             11
                          ..
i8 Base                    5
tC Anniversary Edition     2
tC Base                    1
tC Release Series 6.0      1
xB Base                    1
Name: count, Length: 1898, dtype: int64

Because there are 1898 unique instances within 'model', I manually inspected the column to identify potential issues in need of cleaning. I identified the following steps:<br>
•Standardize title case<br>
•Remove redundancies in model name, eg. repeating model names<br>
•Extract base model and trim into separate columns. Create new column for trim.

In [8]:
#Normalize the case
cars['model'] = cars['model'].str.title()

In [9]:
#Remove redundant patterns-- repeated model names
cars['model_cleaned'] = cars['model'].str.replace(r'(?i)\b(\w+)\b\s+\1\b', r'\1', regex=True)

In [10]:
#Extract base model and trim into separate columns
cars[['base_model', 'trim']] = cars['model'].str.extract(r'^([\w-]+(?:\s[\w-]+)?)\s*(.*)$')

In [11]:
cars.head()

,brand,model,model_year,mileage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,model_cleaned,base_model,trim
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300,Utility Police Interceptor Base,Utility Police,Interceptor Base
1,Hyundai,Palisade Sel,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005,Palisade Sel,Palisade Sel,
2,Lexus,Rx 350 Rx 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,54598,Rx 350 Rx 350,Rx 350,Rx 350
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,15500,Q50 Hybrid Sport,Q50 Hybrid,Sport
4,Audi,Q3 45 S Line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,34999,Q3 45 S Line Premium Plus,Q3 45,S Line Premium Plus


In [12]:
#Drop 'model' and model_cleaned from the original dataframe
cars.drop(['model', 'model_cleaned'], axis=1, inplace=True)

In [13]:
#Move 'base_model' to 2nd column position
base_model = cars.pop('base_model')
cars.insert(1, 'base_model', base_model)

#Move 'trim' to 3rd column position
trim = cars.pop('trim')
cars.insert(2, 'trim', trim)


In [14]:
cars.head()

,brand,base_model,trim,model_year,mileage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police,Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300
1,Hyundai,Palisade Sel,,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005
2,Lexus,Rx 350,Rx 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,54598
3,INFINITI,Q50 Hybrid,Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,15500
4,Audi,Q3 45,S Line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,34999


base_model and trim are still not eliminating all instances of repeated names-- see index 2

In [15]:
# Ensure 'trim' is empty for cases where the model is fully consumed by 'base_model'
cars['trim'] = cars['trim'].where(cars['trim'] != cars['base_model'], '')

In [16]:
cars.head()

,brand,base_model,trim,model_year,mileage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police,Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300
1,Hyundai,Palisade Sel,,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005
2,Lexus,Rx 350,,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,54598
3,INFINITI,Q50 Hybrid,Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,15500
4,Audi,Q3 45,S Line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,34999


### Model year

In [17]:
cars['model_year'].value_counts().sort_index()

model_year
1974      1
1992      1
1993      9
1994      7
1995      6
1996      8
1997      9
1998     11
1999     15
2000     17
2001     34
2002     32
2003     49
2004     60
2005     72
2006     66
2007     98
2008    113
2009     72
2010    100
2011    124
2012    141
2013    158
2014    181
2015    228
2016    268
2017    259
2018    315
2019    297
2020    322
2021    350
2022    354
2023    226
2024      6
Name: count, dtype: int64

This column looks fine data-wise, but after further consideration, year is more of a categorical variable with cars.

In [18]:
#Change model_year to category
cars['model_year'] = cars['model_year'].astype('category')

### Mileage

In [19]:
cars['mileage'].value_counts().sort_index()

mileage
100       1
105       1
115       1
124       1
154       1
         ..
300183    1
315000    1
333361    1
399000    1
405000    1
Name: count, Length: 2818, dtype: int64

It appears everything is in order for this column. But, let's make sure that all the values are integers within the range of 0-405000 (since that is the maximum value listed in the printout above).

In [20]:
#Check if all values are integers within the range
valid_mileage = cars['mileage'].apply(lambda x: isinstance(x, int) and 0 <= x <= 405000)
if valid_mileage.all():
    print('All mileage values are valid.')
else:
    print('There are invalid milage values')

All mileage values are valid.


### Fuel Type

In [21]:
cars['fuel_type'].value_counts().sort_index()

fuel_type
Diesel             116
E85 Flex Fuel      139
Gasoline          3309
Hybrid             194
Plug-In Hybrid      34
not supported        2
–                   45
Name: count, dtype: int64

#### Let's explore the "not supported" and "-" values to understand what they are more clearly.

In [22]:
cars[(cars['fuel_type'] == 'not supported') | (cars['fuel_type'].str.strip() == '–')]

,brand,base_model,trim,model_year,mileage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
130,Chrysler,Pacifica Touring,,2017,87305,–,–,9-Speed A/T,Silver,Black,None reported,Yes,9000
257,Toyota,Land Cruiser,Base,1993,231500,–,–,A/T,White,Gray,None reported,Yes,29500
338,Mazda,Mazda3 S,Grand Touring,2016,85000,–,–,A/T,Black,Black,At least 1 accident or damage reported,Yes,18500
491,Chrysler,Pacifica Touring,,2017,63000,–,–,9-Speed A/T,White,Beige,None reported,Yes,20000
692,Honda,Civic Ex,,1993,58981,–,–,A/T,White,Beige,None reported,Yes,9599
855,Ford,Bronco,,1974,6217,–,–,–,Dark Gray Metallic,–,None reported,Yes,115000
923,Dodge,Challenger R,/T,2010,100100,–,–,Transmission w/Dual Shift Mode,Purple,Black,None reported,Yes,17000
995,Dodge,Challenger R,/T Scat Pack,2019,43000,–,–,6-Speed M/T,Green,Black,None reported,Yes,50000
1063,Mazda,Mazda6 I,Grand Touring,2016,83100,–,–,A/T,White,White,None reported,Yes,17800
1083,Dodge,Challenger R,/T,2010,73000,–,–,A/T,Black,Black,None reported,Yes,17500


The '-' values essentially amount to NaNs. Since there are only 45 of these in a dataset with 4K+ instances, and since fuel_type is likely a factor in the value of vehicle, these rows will be dropped.

After doing some research, the Toyota Mirai is a hydrogen fuel cell vehicle. Let's change the "fuel_type" for both of these columns to "Hydrogen FCV"

In [23]:
#Drop rows where fuel_type is '-'
cars = cars[cars.fuel_type != '-']

#Change 'not supported' to 'Hydrogen FCV'
cars = cars.replace({'not supported': 'Hydrogen FCV'})

cars[cars['fuel_type'] == 'Hydrogen FCV']

,brand,base_model,trim,model_year,mileage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
2894,Toyota,Mirai Limited,,2021,44000,Hydrogen FCV,182.0HP Electric Motor Hydrogen Fuel,A/T,Gray,Black,None reported,Yes,18500
3700,Toyota,Mirai Base,,2016,40000,Hydrogen FCV,151.0HP Electric Motor Hydrogen Fuel,A/T,Silver,Black,None reported,Yes,9500


### Engine

Let's get a sense for what's here:

In [24]:
#number of unique values
cars['engine'].nunique()

1146

In [25]:
#How many of each?
engine_count = cars['engine'].value_counts().sort_index()
print(engine_count)

engine
1.2L I3 12V GDI DOHC Turbo                                           2
1.3L I3 12V GDI DOHC Turbo                                           3
1.3L I3 12V MPFI DOHC Turbo                                          1
1.4L I4 16V GDI DOHC Turbo                                           2
1.5 Liter Turbo                                                      1
                                                                    ..
Nettuno 3L V-6 port/direct injection, DOHC, variable valve contr     1
Standard Range Battery                                               1
V6                                                                   2
V8                                                                   1
–                                                                   45
Name: count, Length: 1146, dtype: int64


In [26]:
#NaNs?
cars['engine'].isnull().sum()

0

No NaNs, but there is that pesky '-' value again. There are 45, so let's drop them.

In [27]:
cars = cars[cars.engine != '-']

In [28]:
cars['engine'].unique()

array(['300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability',
       '3.8L V6 24V GDI DOHC', '3.5 Liter DOHC', ...,
       '136.0HP 1.8L 4 Cylinder Engine Gasoline Fuel',
       '270.0HP 2.0L 4 Cylinder Engine Gasoline Fuel',
       '420.0HP 5.9L 12 Cylinder Engine Gasoline Fuel'], dtype=object)

Some rows list the horsepower of the engine. Let's extract that and create a new column titled 'HP.' Then, we'll see how many of those values we have and decide what to do with them.

In [29]:
#Extract HP from 'engine' column
cars['horsepower'] = cars['engine'].str.extract(r'(\d+(?:\.\d+)?)HP', expand=False)

#Ensure numeric conversion
cars['horsepower'] = pd.to_numeric(cars['horsepower'], errors='coerce')

#Drop missing values
cars = cars.dropna(subset=['horsepower'])

#Convert to integers
cars['horsepower'] = cars['horsepower'].astype(int)

#Remove extracted horsepower from the 'engine' column
cars['engine'] = cars['engine'].str.replace(r'\d+(?:\.\d+)?HP', '', regex=True).str.strip()

#Apply type conversions for mixed values
cars['horsepower'] = cars['horsepower'].astype(int).apply(
    lambda x: int(x) if pd.notna(x) and x.is_integer() else x)

#Confirm the type of each instance
print(cars['horsepower'].apply(type).value_counts())

horsepower
<class 'int'>    3201
Name: count, dtype: int64


In [30]:
#Move 'engine' to 6th column position
horsepower = cars.pop('horsepower')
cars.insert(7, 'horsepower', horsepower)

Units representing engine size also need to be standardized (e.g., '3 L' vs. 3.0L')

In [31]:
#Function to clean engine values
def clean_engine(engine):
    #Handle engine strings one part at a time
    parts = engine.split() #Split string by spaces
    cleaned_parts = []
    for part in parts:
        if re.fullmatch(r'\d+L', part): #Whole num with 'L'
            cleaned_parts.append(part.replace('L', '.0L')) #Convert to '.0L'
        elif re.fullmatch(r'\d+\.\d+L', part): #Decimal with 'L'
            cleaned_parts.append(part) #Leave untouched
        else:
            cleaned_parts.append(part) #Leave other parts untouched
    return ' '.join(cleaned_parts)

#apply function to engine column
cars['engine_cleaned'] = cars['engine'].apply(clean_engine)

In [32]:
#Extract engine size(#.#L) from 'engine_cleaned' column
cars['engine_size'] = cars['engine_cleaned'].str.extract(r'(\d+\.\d+L)', expand=False)

#Convert to numeric: Remove 'L' and covert to float
cars['engine_size'] = cars['engine_size'].str.replace('L', '', regex=False).astype(float)

In [33]:
#Move 'engine_size' to 7th column
engine_size = cars.pop('engine_size')
cars.insert(8, 'engine_size', engine_size)

In [34]:
engine_cleaned = cars.pop('engine_cleaned')
cars.insert(6, 'engine_cleaned', engine_cleaned)

In [35]:
#Drop the original 'engine' column
cars.drop(columns = ['engine'], inplace=True)

In [36]:
#Remove engine size from 'engine' column
cars['engine_cleaned'] = cars['engine_cleaned'].str.replace(r'\b\d+\.\d+L\b', '', regex=True).str.strip()

In [37]:
#Rename 'engine_cleaned' to 'engine'
cars.rename(columns={'engine_cleaned':'engine'}, inplace=True)

In [38]:
cars.head()

,brand,base_model,trim,model_year,mileage,fuel_type,engine,horsepower,engine_size,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police,Interceptor Base,2013,51000,E85 Flex Fuel,V6 Cylinder Engine Flex Fuel Capability,300,3.7,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300
3,INFINITI,Q50 Hybrid,Sport,2015,88900,Hybrid,V6 Cylinder Engine Gas/Electric Hybrid,354,3.5,7-Speed A/T,Black,Black,None reported,Yes,15500
6,Audi,S3 2,.0T Premium Plus,2017,84000,Gasoline,4 Cylinder Engine Gasoline Fuel,292,2.0,6-Speed A/T,Blue,Black,None reported,Yes,31000
7,BMW,740 Il,,2001,242000,Gasoline,8 Cylinder Engine Gasoline Fuel,282,4.4,A/T,Green,Green,None reported,Yes,7300
8,Lexus,Rc 350,F Sport,2021,23436,Gasoline,V6 Cylinder Engine Gasoline Fuel,311,3.5,6-Speed A/T,Black,Black,None reported,Yes,41927


In [39]:
#Extract engine_type (e.g. "V6 Cylinder Engine")
cars['engine_type'] = cars['engine'].str.extract(r'^(.*Engine)\b', expand=False)

#Extract fuel_type (e.g., "Flex Fuel Capability"
cars['fuel_type'] = cars['engine'].str.extract(r'Engine\s+(.*)$', expand=False)

In [40]:
#Fill unknown values in both columns
cars['engine_type'] = cars['engine_type'].fillna('Unknown Engine')
cars['fuel_type'] = cars['fuel_type'].fillna('Unknown Fuel Type')

In [41]:
#Move engine_type to 5th index position
engine_type = cars.pop('engine_type')
cars.insert(5, 'engine_type', engine_type)

In [42]:
#Drop 'engine' completely-- we've segmented all the data out
cars.drop(columns=['engine'], inplace=True)

### Transmission

In [43]:
cars['transmission'].nunique()

18

In [44]:
cars['transmission'].value_counts().sort_index()

transmission
1-Speed A/T                         61
10-Speed A/T                       119
2-Speed A/T                          3
4-Speed A/T                         68
5-Speed A/T                         84
5-Speed M/T                         53
6-Speed A/T                        360
6-Speed M/T                        243
7-Speed A/T                        209
7-Speed M/T                         10
8-Speed A/T                        405
9-Speed A/T                         72
A/T                               1007
Automatic                            2
CVT Transmission                    62
M/T                                 40
Transmission Overdrive Switch        7
Transmission w/Dual Shift Mode     396
Name: count, dtype: int64

Several steps will be taken to clean the 'transmission' column:

•Count 'Transmission w/Dual Shift Mode' as 'A/T' value, but first create a binary column "dual_shift_mode" to highlight the vehicles that have this feature.

•Extract whehter the transmssion is automatic or manual. Create a new column with this information called 'transmission_type'

•Extract the number of speeds in the transmission. Create a new column titled "num_speeds"

•Cars with overdrive switches are typically automatic tranmissions, so those 7 values will be combined into the 'A/T' values


In [45]:
#Create binary column for 'dual_shift_mode'
cars['dual_shift_mode'] = cars['transmission'].str.contains(r'Dual Shift Mode', regex=False).astype(int)

#Combine 'Overdrive Switch' and 'Dual Shift Mode' into 'A/T'
cars['transmission'] = cars['transmission'].replace({'Transmission Overdrive Switch': 'A/T',
                                                     'Transmission w/Dual Shift Mode': 'A/T',
                                                    'Automatic': 'A/T'})

In [46]:
#Shorten 'CVT Transmission
cars['transmission_type'] = cars['transmission'].replace('CVT Transmission', 'CVT', regex=False)

In [47]:
# Extract 'transmission_type' as either 'A/T' or 'M/T'
def extract_transmission_type(transmission):
    if 'A/T' in transmission:
        return 'A/T'
    elif 'M/T' in transmission:
        return 'M/T'
    else:
        return 'Unknown'

cars['transmission_type'] = cars['transmission'].map(extract_transmission_type)

In [48]:
#Extract 'number_of_speeds' from 'transmission'
cars['number_of_speeds'] = cars['transmission'].str.extract(r'(\d+)-Speed', expand=False)
cars['number_of_speeds'] = cars['number_of_speeds'].fillna('Unknown')

In [49]:
#Any remaining values in 'transmission' that need to be cleaned?
cars['transmission'].value_counts()

transmission
A/T                 1412
8-Speed A/T          405
6-Speed A/T          360
6-Speed M/T          243
7-Speed A/T          209
10-Speed A/T         119
5-Speed A/T           84
9-Speed A/T           72
4-Speed A/T           68
CVT Transmission      62
1-Speed A/T           61
5-Speed M/T           53
M/T                   40
7-Speed M/T           10
2-Speed A/T            3
Name: count, dtype: int64

It looks like all the cleaning that is needed has been done. Now, I will re-arrange the order of the columns so the flow similarly to how the df was structured after cleaning 'engine'.

In [50]:
#Drop the original 'transmission' column:
cars.drop(columns=['transmission'], inplace=True)

In [51]:
#Move 'transmission_type' to the 9th column position
transmission_type = cars.pop('transmission_type')
cars.insert(10, 'transmission_type', transmission_type)

In [52]:
#Move 'number of speeds' to the 10th column position
number_of_speeds = cars.pop('number_of_speeds')
cars.insert(11, 'number_of_speeds', number_of_speeds)

In [53]:
#Move 'dual_shift_mode' to the 11th column position
dual_shift_mode = cars.pop('dual_shift_mode')
cars.insert(12, 'dual_shift_mode', dual_shift_mode)

### Exterior color

In [54]:
cars['ext_col'].value_counts()

ext_col
Black                      817
White                      749
Gray                       460
Silver                     347
Blue                       326
Red                        236
Green                       67
Brown                       43
Gold                        41
Beige                       37
Orange                      36
Yellow                      28
Purple                      11
Silky Silver                 1
Pink                         1
Midnight Black Metallic      1
Name: count, dtype: int64

Silky Silver and Midnight Black Metallic can be simplified to 'Silver' and 'Metallic'.

In [55]:
cars['ext_col'] = cars['ext_col'].replace(['Silky Silver', 'Midnight Black Metallic'], ['Silver', 'Black'])

In [56]:
cars['ext_col'].value_counts()

ext_col
Black     818
White     749
Gray      460
Silver    348
Blue      326
Red       236
Green      67
Brown      43
Gold       41
Beige      37
Orange     36
Yellow     28
Purple     11
Pink        1
Name: count, dtype: int64

### Accident

In [57]:
cars['accident'].value_counts()

accident
None reported                             2234
At least 1 accident or damage reported     884
Name: count, dtype: int64

In [58]:
cars['accident'].isna().sum()

83

Since there are only two values-- essentially a Y/N-- and we eventually want to pass this information into ML models, this column will be converted to a binary and the name will be changed. The new column name will be 'accident-damage_reported.' The values will be 0, representing 'No' and 1 representing 'Yes'. There will also be an 'Unknown' value to accomodate the NA values found above. Finally, the original 'accident' column will be dropped and replaced in the same index position with the new 'accident-damage_reported' column.

In [59]:
cars['accident_damage_reported'] = cars['accident'].apply(
    lambda x: 0 if pd.notna(x) and 'None reported' in str(x) else (1 if pd.notna(x) else 'Unknown'))

In [61]:
accident_damage_reported = cars.pop('accident_damage_reported')
cars.insert(14, 'accident_damage_reported', accident_damage_reported)

In [63]:
cars.drop(columns=['accident'], inplace=True)

### Clean title

In [63]:
cars['clean_title'].value_counts()

clean_title
Yes    3116
Name: count, dtype: int64

In [65]:
cars['clean_title'].isna().sum()

85

Similarly to the 'accident' column, this column can be converted to a Y/N and 'Unknown' column.

In [66]:
cars['clean_title'] = cars['clean_title'].apply(
    lambda x: 1 if pd.notna(x) and 'Yes' in str(x) else (0 if pd.notna(x) else 'Unknown'))

In [67]:
cars['clean_title'].value_counts()

clean_title
1          3116
Unknown      85
Name: count, dtype: int64

### Price

In [68]:
cars['price'].dtype

dtype('int64')

In [70]:
cars['price'].min()

2000

In [71]:
cars['price'].max()

2954083

In [73]:
#Check if all values in 'price' are integers within the range of min and max values
is_valid = cars['price'].apply(lambda x: isinstance(x, int) and 2000 <= x <= 2954083).all()

print(f"All values in 'price' are valid integers within the range: {is_valid}")

All values in 'price' are valid integers within the range: True


### Export the cleaned df

In [75]:
cars.to_csv('used_cars_cleaned.csv', index=False)